In [1]:
from time import time
import matplotlib.pyplot as plt
import sys

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import random_projection
from sklearn.utils.fixes import loguniform
from tqdm import tqdm
import numpy as np
import pickle

In [2]:
def get_data():
    from datamodule import ImageWeightsModule
    import torch
    all_data_loader = ImageWeightsModule("lora_dataset_creator/lora_dataset", 1, augment_training=False, val_split=0).train_dataloader()
    X = None
    Y = None
    for x, y in tqdm(all_data_loader):
        if X is None:
            X = x
        X = torch.cat((X, x), dim=0)
        if Y is None:
            Y = y
        Y = torch.cat((Y, y), dim=0)
    return X, np.array(Y)

X, Y = get_data()

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


filter_files: 335 approved - 0 rejected


Loading all data for PCA: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 335/335 [00:33<00:00,  9.99it/s]


in = (325, 509248)
done in 14.012s
Train loss:
transform done in 0.516s
out = (325, 100)
inverse_transform done in 0.624s
loss = 0.024692270904779434
Val loss:
transform done in 0.078s
out = (10, 100)
inverse_transform done in 0.076s
loss = 0.01966654136776924
X_transfomed_old [[-3.34182382e-01 -2.14696437e-01 -6.38714075e-01 ... -2.76692066e-04
  -7.57627003e-03 -8.52678064e-03]
 [-1.32705018e-01 -6.90683872e-02 -1.84550151e-01 ... -1.49323605e-02
  -1.58650856e-02 -4.85605095e-03]
 [-2.01959312e-01 -3.06299161e-02 -1.54938951e-01 ...  1.37721561e-02
   5.73940109e-03 -2.33296258e-03]
 ...
 [-2.52332747e-01 -1.29932478e-01 -6.87159657e-01 ... -4.44572978e-03
   3.70202539e-03 -4.24425397e-03]
 [-2.52354354e-01 -1.29704192e-01 -6.87345088e-01 ... -3.72816063e-03
   3.65823740e-03 -3.87487980e-03]
 [-3.23834598e-01 -2.97831106e+00  5.43959475e+00 ...  9.36123368e-04
  -3.08727240e-03 -4.58086215e-05]]
X_transfomed_old 2 [[-3.34182382e-01 -2.14696437e-01 -6.38714075e-01 ... -2.76692066e-

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 335/335 [00:09<00:00, 35.24it/s]


In [3]:
scaler = StandardScaler()
Y_scaled = scaler.fit_transform(Y)
Y_scaled = Y
Y_scaled_val = Y_scaled[:10, :]
Y_scaled = Y_scaled[10:, :]
print(Y_scaled.shape)

(326, 100)


In [ ]:
@np.printoptions(suppress=True)
def test_pca(x):
    t0 = time()
    z = pca.transform(x)
    print("transform done in %0.3fs" % (time() - t0))
    print(f"out = {z.shape}")
    t0 = time()
    x_hat = pca.inverse_transform(z)
    print("inverse_transform done in %0.3fs" % (time() - t0))
    print(f"loss = {abs(x_hat - x).mean()}")

In [ ]:
n_components = 100
t0 = time()
print(f"in = {Y_scaled.shape}")
pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True, random_state=5).fit(Y_scaled)
print("done in %0.3fs" % (time() - t0))

In [ ]:
test_pca(Y_scaled)
test_pca(Y_scaled_val)

In [ ]:
eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()

In [ ]:
print("Fitting the classifier to the training set")
t0 = time()
param_grid = {
    "C": loguniform(1e3, 1e5),
    "gamma": loguniform(1e-4, 1e-1),
}
clf = RandomizedSearchCV(
    SVC(kernel="rbf", class_weight="balanced"), param_grid, n_iter=10
)
clf = clf.fit(X_train_pca, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

In [ ]:
print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred, target_names=target_names))
ConfusionMatrixDisplay.from_estimator(
    clf, X_test_pca, y_test, display_labels=target_names, xticks_rotation="vertical"
)
plt.tight_layout()
plt.show()